In [2]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

### Basic autograd example 1

In [8]:
x = Variable(torch.Tensor([1]),requires_grad=True)
w = Variable(torch.Tensor([2]),requires_grad=True)
b = Variable(torch.Tensor([3]),requires_grad=True)
y = w * x + b
y.backward()
print(x.grad) # partial derivative
print(w.grad)
print(b.grad)

Variable containing:
 2
[torch.FloatTensor of size 1]

Variable containing:
 1
[torch.FloatTensor of size 1]

Variable containing:
 1
[torch.FloatTensor of size 1]




### Basic autograd example 2

In [9]:
# create tensors
x = Variable(torch.randn(5,3))
y = Variable(torch.randn(5,2))

# build a linear layer
linear = nn.Linear(3,2)
print('w: ',linear.weight)
print('b: ',linear.bias)

# build loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(),lr=0.01)

# forward propagation
pred = linear(x)

# compute loss
loss = criterion(pred,y)
print('loss: ',loss.data[0])

# backpropagation
loss.backward()

# print out the gradients
print('dL/dw: ',linear.weight.grad)
print('dL/db: ',linear.bias.grad)

# 1-step Optimization(gradient descent)
optimizer.step()

# you can also do optimization as the low level as shown below
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.weight.data.sub_(0.01 * linear.bias.grad.data)

# print out the loss after optimization
pred = linear(x)
loss = criterion(pred,y)
print('loss after 1 step optimization: ',loss.data[0])


('w: ', Parameter containing:
-0.2663  0.5208  0.1099
-0.1311 -0.5619 -0.4654
[torch.FloatTensor of size 2x3]
)
('b: ', Parameter containing:
 0.2685
-0.2168
[torch.FloatTensor of size 2]
)
('loss: ', 2.511803388595581)
('dL/dw: ', Variable containing:
-0.9917  0.5712 -0.3708
-0.7451 -0.9118 -0.1843
[torch.FloatTensor of size 2x3]
)
('dL/db: ', Variable containing:
 0.7504
-0.5308
[torch.FloatTensor of size 2]
)
('loss after 1 step optimization: ', 2.474860191345215)


### Numpy bridge

In [10]:
a = np.array([[2,3],[4,5]])
b = torch.from_numpy(a) # convert numpy array to torch tensor
c = b.numpy(b) # convert torch tensor to numpy array

### Implementing the input pipline

In [14]:
# Download and construct dataset
train_dataset = dsets.CIFAR10(root = '../data/',train=True,transform=transforms.ToTensor(),download=True)
# Select one data pair (read data from disk)
image,label = train_dataset[0]
print(image.size())
print(label)
# Data Loader (this privides queue and thread in a very simple way)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=100,shuffle=True,num_workers=2)

# When iteration starts, queue and thread start to load dataset
data_iter = iter(train_loader)

# Mini-batch images and labels
images,labels = data_iter.next()

# Actual usage of data loader is as below
for images,labels in train_loader:
    # Your training code will be written here
    pass

Files already downloaded and verified
torch.Size([3, 32, 32])
6


### Input pipline for custom dataset

In [ ]:
# You should build custom dataset as below
class CustomDataset(data.Dataset):
    def __init__(self):
        # TODO
        # 1.Initialize file path or list of file names,
        pass
    def __getitem__(self,index):
        # TODO
        # 1. Read one data from file(e.g. using numpy.fromfile,PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g, image and label)
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0
# Then, you can just use prebuilt torch's data loader
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                          batch_size = 100,
                                          shuffle = TRUE，
                                          num_workers = 2)
        

### Using pretrained resnet

In [ ]:
# Download and load pretrained resnet
resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetune only top layer of the model.
for param in resnet.parameters():
    param.requires_grad = False

# Replace top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features,100) # 100 is for example.

# For test.
images = Variable(torch.randn(10,3,256,256))
outputs = resnet(images)
print(outputs.size()) # (10,100)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/g1002/.torch/models/resnet18-5c106cde.pth
 92%|█████████▏| 43171840/46827520 [31:53<02:38, 23046.25it/s]

### Save and load the model

In [ ]:
# Save and load the entire model
torch.save(resnet,'model.pkl')
model = torch.load('model.pkl')

# Save and load only the model parameters(recommended)
torch.save(resnet.state_dict(),'params.pkl')
resnet.load_state_dict(torch.load('params.pkl'))